## Install necessary libraries

In [1]:
!pip install beautifulsoup4 requests sqlalchemy selenium


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Import libraries

In [7]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from sqlalchemy import create_engine
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time


## Next we test for response 200

In [8]:
url='https://www.aliexpress.com/w/wholesale-laptops.html?g=y&SearchText=laptops'

r = requests.get(url)

print(r.status_code)

200


# After donwloading and extracting the google chrome driver in the file path of our folder, we then configure selenium chrome driver options

In [9]:
# Configure Selenium ChromeDriver options
options = Options()
options.headless = True
service = Service(executable_path=r'chromedriver-win64\chromedriver.exe')

# Initialize the webdriver
driver = webdriver.Chrome(service=service,options=options)

#Define the URL
url= 'https://www.aliexpress.com/w/wholesale-laptops.html?g=y&SearchText=laptops'

# Use Selenium to open the page


#Wait for the Dynamic content to load
time.sleep(10)

# Get the page source and close the browser
page_source= driver.page_source
driver.quit()

#  Extraction Layer

In [10]:
# Configure Selenium ChromeDriver options
options = Options()
options.headless = True
service = Service(executable_path=r'chromedriver-win64\chromedriver.exe')

# Initialize the webdriver
driver = webdriver.Chrome(service=service,options=options)

# List to store extracted data
product_names=[]
prices=[]
store_link=[]
store_names=[]
shipping_prices=[]
extra_discounts=[]
item_sold=[]
original_price=[]
shipping_free_status=[]
sales_prices=[]

# Assuming you know the total number of pages
total_pages=60

#so lets use selenium to stabilize the page and use beautiful soup to extra_discounts

for  page_number in range(1,total_pages + 1):
    url='https://www.aliexpress.com/w/wholesale-laptops.html?g=y&SearchText=laptops'
    driver.get(url)
    time.sleep(10)

    # Now use beautifulsoup to parse the loaded page_source
    # Get the page source 
    #page_source= driver.page_source
    soup= BeautifulSoup(driver.page_source,'html.parser')

    #Next lets adjust the class selectors based on the current website structure
    laptops = soup.find_all('div',class_ ='multi--modalContext--1Hxqhwi')

    for laptop in laptops:
        #product name
        product_name=laptop.find('div',class_ ='multi--title--G7dOCj3').text

        #price
        
        price=laptop.find('div', class_ ='multi--price-sale--U-S0jtj').text.replace('￡','').replace(',','')

        #store name
        store_name=laptop.find('a', class_ ='cards--storeLink--XkKUQFS').text

        #store_link(becuase we are not sure if all the stores have a link, we use the try and except method just incase its not there)
        try:
            store_links= laptop.find('a', class_ ='cards--storeLink--XkKUQFS')['href']
        except (TypeError,KeyError):
            store_link=''

        #shipping_price(here we will also use the try and except clause)
        try:
            shipping_price=laptop.find('span',class_ ='tag--text--1BSEXVh tag--textStyle--3dc7wLU multi--serviceStyle--1Z6RxQ4').text
        except AttributeError:
            shipping_price=''

        # extra discount
        try:
            extra_discount=laptop.find('span', class_ ='tag--text--1BSEXVh tag--textStyle--3dc7wLU multi--superStyle--1jUmObG').text
        except AttributeError:
            extra_discount=''

        # Item sold
        try:
            item_sold_text=laptop.find('span', class_ ='multi--trade--Ktbl2jB').text
            item_sold_value=item_sold_text.split()[0] if item_sold_text else ''
        except AttributeError:
            item_sold_value=''

        # Append data to list

        product_names.append( product_name)
        prices.append(price)
        store_link.append(store_links) 
        store_names.append( store_name)     
        shipping_prices.append(shipping_price)
        extra_discounts.append(extra_discount)
        item_sold.append(item_sold_value)

        # Now lets go ahead to extract the remaining additional information
        #original_prices
        former_price=laptop.find('div',class_ ='multi--price-original--1zEQqOK').text.replace('￡','').replace(',','')
        original_price.append(former_price)
        #original_price.append(former_price[0].text if former_price else '')
        
        #shipping_free_status
        free_shipping=laptop.find_all('div',class_ = 'multi--serviceContainer--3vRdzWN')
        ship_free=free_shipping[0].text.strip() if free_shipping else ''
        shipping_free_status.append(ship_free)

        #sales_prices=[]
        

# Once all the pages information are extracted,quit the driver
driver.quit()


# CREATE A DATAFRAME

# Next lets create a dataframe for our data collected above
# but first lets put them into a dictionary
data={
    'Product_Name': product_names,
    'Prices': prices,
    'Store_Link':store_link,
    'Store_Name': store_name,
    'Shipping_Prices':shipping_prices,
    'Extra_Discounts':extra_discounts,
    'Item_Sold':item_sold,
    'Original_Prices':original_price,
    'Shipping_free_status':shipping_free_status,
}

# Next we put this dictionary into a dataframe
df=pd.DataFrame(data)



In [11]:
#display our top records
display(df.head())

,Product_Name,Prices,Store_Link,Store_Name,Shipping_Prices,Extra_Discounts,Item_Sold,Original_Prices,Shipping_free_status
0,"14.1"" Ultra Slim Laptop 16GB RAM 2TB SSD Intel...",147.14,//www.aliexpress.com/store/1102996501,Quality Electronics Store,Free shipping,Extra 2% off with coins,500+,,Free shipping
1,"Portable 15.6"" Laptop 32GB Ram DDR4 2TB SSD Wi...",196.66,//www.aliexpress.com/store/1103309564,Quality Electronics Store,Free shipping,Extra 2% off with coins,600+,,Free shipping
2,Adreamer LeoBook13 Laptop 8GB RAM 1TB SSD Comp...,116.94,//www.aliexpress.com/store/1100347101,Quality Electronics Store,Free shipping,Extra 5% off with coins,354,,Free shipping
3,15.6 Inch Laptop 32GB Ram 2TB SSD Windows 11 N...,161.67,//www.aliexpress.com/store/1103515369,Quality Electronics Store,Free shipping,,25,175.29,Free shipping
4,Teclast F7 Plus 2 14.1 Inch Laptop Intel Celer...,143.61,//www.aliexpress.com/store/912427260,Quality Electronics Store,Free shipping,,10,362.08,Free shipping


In [18]:
df.shape

(608, 9)

# now lets run for 60 pages of Ali express and increase the timing to 3600s
change total_pages =from  1 to 60
and time.sleep from 10 to 3600, but it is advisable to leave time at 10 seconds

In [13]:
# Save the raw data to csv
df.to_csv('Aliexpresslaptops.csv',index=False)

## Data Transformation

In [19]:
df.columns

Index(['Product_Name', 'Prices', 'Store_Link', 'Store_Name', 'Shipping_Prices',
       'Extra_Discounts', 'Item_Sold', 'Original_Prices',
       'Shipping_free_status'],
      dtype='object')

In [36]:
ali_df=pd.DataFrame(df)

ali_df

,Product_Name,Prices,Store_Link,Store_Name,Shipping_Prices,Extra_Discounts,Item_Sold,Original_Prices,Shipping_free_status
0,"14.1"" Ultra Slim Laptop 16GB RAM 2TB SSD Intel...",147.14,//www.aliexpress.com/store/1102996501,Quality Electronics Store,Free shipping,Extra 2% off with coins,500+,,Free shipping
1,"Portable 15.6"" Laptop 32GB Ram DDR4 2TB SSD Wi...",196.66,//www.aliexpress.com/store/1103309564,Quality Electronics Store,Free shipping,Extra 2% off with coins,600+,,Free shipping
2,Adreamer LeoBook13 Laptop 8GB RAM 1TB SSD Comp...,116.94,//www.aliexpress.com/store/1100347101,Quality Electronics Store,Free shipping,Extra 5% off with coins,354,,Free shipping
3,15.6 Inch Laptop 32GB Ram 2TB SSD Windows 11 N...,161.67,//www.aliexpress.com/store/1103515369,Quality Electronics Store,Free shipping,,25,175.29,Free shipping
4,Teclast F7 Plus 2 14.1 Inch Laptop Intel Celer...,143.61,//www.aliexpress.com/store/912427260,Quality Electronics Store,Free shipping,,10,362.08,Free shipping
...,...,...,...,...,...,...,...,...,...
603,14.1 inch portable laptop Intel N3700 16GB RAM...,154.21,//www.aliexpress.com/store/1103350119,Quality Electronics Store,Free shipping,Extra 5% off with coins,50,198.82,Free shipping
604,"Ultra Slim Laptop 14.1"" 16GB RAM 2TB SSD Intel...",149.72,//www.aliexpress.com/store/1103309564,Quality Electronics Store,Free shipping,Extra 2% off with coins,900+,,Free shipping
605,"Portable Laptop 15.6"" 32GB Ram DDR4 2TB SSD Wi...",198.25,//www.aliexpress.com/store/1103309564,Quality Electronics Store,Free shipping,Extra 2% off with coins,36,,Free shipping
606,15.6' Office Study Laptop Computer Windows 11 ...,161.67,//www.aliexpress.com/store/1103515369,Quality Electronics Store,Free shipping,,37,175.29,Free shipping


In [41]:
ali_df.columns

Index(['Product_Name', 'Prices', 'Store_Link', 'Store_Name', 'Shipping_Prices',
       'Extra_Discounts', 'Item_Sold', 'Original_Prices',
       'Shipping_free_status'],
      dtype='object')

Now lets make the different tables
first we make the product tables

In [46]:
Product_df=ali_df[['Product_Name','Prices','Store_Name','Store_Link']].copy().drop_duplicates().reset_index(drop=True)

Product_df.head()

,Product_Name,Prices,Store_Name,Store_Link
0,"14.1"" Ultra Slim Laptop 16GB RAM 2TB SSD Intel...",147.14,Quality Electronics Store,//www.aliexpress.com/store/1102996501
1,"Portable 15.6"" Laptop 32GB Ram DDR4 2TB SSD Wi...",196.66,Quality Electronics Store,//www.aliexpress.com/store/1103309564
2,Adreamer LeoBook13 Laptop 8GB RAM 1TB SSD Comp...,116.94,Quality Electronics Store,//www.aliexpress.com/store/1100347101
3,15.6 Inch Laptop 32GB Ram 2TB SSD Windows 11 N...,161.67,Quality Electronics Store,//www.aliexpress.com/store/1103515369
4,Teclast F7 Plus 2 14.1 Inch Laptop Intel Celer...,143.61,Quality Electronics Store,//www.aliexpress.com/store/912427260


Lets make the product_id column

In [47]:
#Product Table
#create a product_id
Product_df.index.name='Product_id'
Product_df=Product_df.reset_index()






In [50]:
Product_df.head()

,Product_id,Product_Name,Prices,Store_Name,Store_Link
0,0,"14.1"" Ultra Slim Laptop 16GB RAM 2TB SSD Intel...",147.14,Quality Electronics Store,//www.aliexpress.com/store/1102996501
1,1,"Portable 15.6"" Laptop 32GB Ram DDR4 2TB SSD Wi...",196.66,Quality Electronics Store,//www.aliexpress.com/store/1103309564
2,2,Adreamer LeoBook13 Laptop 8GB RAM 1TB SSD Comp...,116.94,Quality Electronics Store,//www.aliexpress.com/store/1100347101
3,3,15.6 Inch Laptop 32GB Ram 2TB SSD Windows 11 N...,161.67,Quality Electronics Store,//www.aliexpress.com/store/1103515369
4,4,Teclast F7 Plus 2 14.1 Inch Laptop Intel Celer...,143.61,Quality Electronics Store,//www.aliexpress.com/store/912427260


In [53]:
## Lets rename one of the column names
Product_df.rename(columns={'Store_Name':'Store_Namez'}, inplace=True)

Product_df

,Product_id,Product_Name,Prices,Store_Namez,Store_Link
0,0,"14.1"" Ultra Slim Laptop 16GB RAM 2TB SSD Intel...",147.14,Quality Electronics Store,//www.aliexpress.com/store/1102996501
1,1,"Portable 15.6"" Laptop 32GB Ram DDR4 2TB SSD Wi...",196.66,Quality Electronics Store,//www.aliexpress.com/store/1103309564
2,2,Adreamer LeoBook13 Laptop 8GB RAM 1TB SSD Comp...,116.94,Quality Electronics Store,//www.aliexpress.com/store/1100347101
3,3,15.6 Inch Laptop 32GB Ram 2TB SSD Windows 11 N...,161.67,Quality Electronics Store,//www.aliexpress.com/store/1103515369
4,4,Teclast F7 Plus 2 14.1 Inch Laptop Intel Celer...,143.61,Quality Electronics Store,//www.aliexpress.com/store/912427260
5,5,"Ultra Slim Laptop 14.1"" 16GB RAM 2TB SSD Intel...",149.72,Quality Electronics Store,//www.aliexpress.com/store/1103309564
6,6,14.1 inch portable laptop Intel N3700 16GB RAM...,154.21,Quality Electronics Store,//www.aliexpress.com/store/1103350119
7,7,"Portable Laptop 15.6"" 32GB Ram DDR4 2TB SSD Wi...",198.25,Quality Electronics Store,//www.aliexpress.com/store/1103309564
8,8,FIREBAT A14 Laptop Intel N5095 14.1 Inch 16GB ...,196.72,Quality Electronics Store,//www.aliexpress.com/store/1103843268
9,9,2024 New Laptop Computer Windows 11 Pro Ultra ...,148.33,Quality Electronics Store,//www.aliexpress.com/store/1102997522


In [54]:
ali_df.columns

Index(['Product_Name', 'Prices', 'Store_Link', 'Store_Name', 'Shipping_Prices',
       'Extra_Discounts', 'Item_Sold', 'Original_Prices',
       'Shipping_free_status'],
      dtype='object')

Next lets make the Discount table

In [55]:
Discount_df=ali_df[['Prices','Original_Prices','Extra_Discounts']].copy().drop_duplicates().reset_index(drop=True)

Discount_df.head()

,Prices,Original_Prices,Extra_Discounts
0,147.14,,Extra 2% off with coins
1,196.66,,Extra 2% off with coins
2,116.94,,Extra 5% off with coins
3,161.67,175.29,
4,143.61,362.08,


In [56]:
Discount_df.index.name='Discount_id'
Discount_df=Discount_df.reset_index()

In [ ]:
## Next lets do more cleaning by removing currency symbols and commas then we comvert to float the original_price column


